In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import os

2022-09-29 17:11:15.990209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-29 17:11:16.326361: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-29 17:11:17.181057: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/can/.local/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda-11.2/lib64:
2022-09-29 17:11:17.181160: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not

In [2]:
BASE_PATH = '../gtsrb/'
TRAIN_PATH = BASE_PATH + 'Train'


data = []
labels = []

NUM_CLASSES = len(os.listdir(TRAIN_PATH))
IMG_HEIGHT = 48
IMG_WIDTH = 48
IMG_CHAN = 3

img_shape = (IMG_HEIGHT, IMG_WIDTH, IMG_CHAN)

NUM_CATEGORIES = len(os.listdir(TRAIN_PATH))
for category in range(NUM_CLASSES):
        categories = os.path.join(TRAIN_PATH, str(category))
        for img in os.listdir(categories):
            image = Image.open(os.path.join(categories, img))
            image = image.resize((IMG_HEIGHT,IMG_WIDTH))
            image = np.array(image)
            image = image/255.0
            data.append(image)

In [7]:
# Convert lists to NumPy arrays
data = np.array(data)
x_train, x_test = train_test_split(data,test_size=0.2, random_state=16)
print(x_train.shape, x_test.shape)

(31367, 48, 48, 3) (7842, 48, 48, 3)


In [8]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 48, 48, 3))
x_test = np.reshape(x_test, (len(x_test), 48, 48, 3))

In [13]:
# Create the Encoder and Decoder
#pass the gray scale input image of size(28,28,1)
inputs = tf.keras.Input(shape=(48, 48, 3), name='input_layer')

# Conv Block 1 -> BatchNorm->leaky Relu
encoded = tf.keras.layers.Conv2D(32, kernel_size=(3,3), strides= 1, padding='same', name='conv_1')(inputs)
encoded = tf.keras.layers.BatchNormalization(name='batchnorm_1')(encoded)
encoded = tf.keras.layers.LeakyReLU(name='leaky_relu_1')(encoded)

# Conv Block 2 -> BatchNorm->leaky Relu
encoded = tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides= 2, padding='same', name='conv_2')(encoded)
encoded = tf.keras.layers.BatchNormalization(name='batchnorm_2')(encoded)
encoded = tf.keras.layers.LeakyReLU(name='leaky_relu_2')(encoded)

# Conv Block 3 -> BatchNorm->leaky Relu
encoded = tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides=2, padding='same', name='conv_3')(encoded)
encoded = tf.keras.layers.BatchNormalization(name='batchnorm_3')(encoded)
encoded = tf.keras.layers.LeakyReLU(name='leaky_relu_3')(encoded)

#Decoder
# DeConv Block 1-> BatchNorm->leaky Relu
decoded = tf.keras.layers.Conv2DTranspose(64, (3,3), strides= 1, padding='same',name='conv_transpose_1')(encoded)
decoded = tf.keras.layers.BatchNormalization(name='batchnorm_4')(decoded)
decoded = tf.keras.layers.LeakyReLU(name='leaky_relu_4')(decoded)

# DeConv Block 2-> BatchNorm->leaky Relu
decoded = tf.keras.layers.Conv2DTranspose(64, (3,3), strides= 2, padding='same', name='conv_transpose_2')(decoded)
decoded = tf.keras.layers.BatchNormalization(name='batchnorm_5')(decoded)
decoded = tf.keras.layers.LeakyReLU(name='leaky_relu_5')(decoded)

# DeConv Block 3-> BatchNorm->leaky Relu
decoded = tf.keras.layers.Conv2DTranspose(32, (3,3), 2, padding='same', name='conv_transpose_3')(decoded)
decoded = tf.keras.layers.BatchNormalization(name='batchnorm_6')(decoded)
decoded = tf.keras.layers.LeakyReLU(name='leaky_relu_6')(decoded)

# output
outputs = tf.keras.layers.Conv2DTranspose(1, 3, 1,padding='same', activation='sigmoid', name='conv_transpose_4')(decoded)

hello
hello
hello


In [ ]:
ae = tf.keras.models.Sequential()
ae.add(tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), activation='relu', input_shape=img_shape))
ae.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation='relu'))
ae.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
ae.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
ae.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
ae.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
ae.add(tf.keras.layers.Flatten())
ae.add(tf.keras.layers.Dense(1024, activation='relu'))
ae.add(tf.keras.layers.Dense(256, activation='relu'))
ae.add(tf.keras.layers.Dense(1024, activation='relu'))
ae.add(tf.keras.layers.Dense(5184, activation='relu'))
ae.add(tf.keras.layers.Reshape((9, 9, 64)))
ae.add(tf.keras.layers.UpSampling2D(size=(2, 2)))
ae.add(tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), activation='relu'))
ae.add(tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=(3, 3), activation='relu'))
ae.add(tf.keras.layers.UpSampling2D(size=(2, 2)))
ae.add(tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=(3,3), activation='relu'))
ae.add(tf.keras.layers.Conv2DTranspose(filters=3, kernel_size = (3,3), activation = 'relu'))

In [10]:
def SSIMLoss(y_true, y_pred):
    return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred,1.0))

In [14]:
autoencoder = tf.keras.Model(inputs, outputs)
optimizer = tf.keras.optimizers.Adam(lr = 0.0005)
autoencoder.compile(optimizer=optimizer, loss=SSIMLoss)

In [15]:
hist=autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test)
                )

Epoch 1/10


ValueError: in user code:

    File "/home/can/.local/lib/python3.8/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/tmp/ipykernel_160103/795528271.py", line 2, in SSIMLoss  *
        return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred,1.0))

    ValueError: Shapes (48, 48, 3) and (144, 144, 1) are incompatible
